# Imports

In [1]:
from astropy.io import fits
from astropy import wcs
from astropy.table import Table, Column
from bokeh.plotting import figure, output_file, show
from bokeh.layouts import gridplot
from bokeh.io import output_notebook, reset_output
import numpy as np
import copy

# Parameters

In [2]:
# Path
path = '/Users/nflagey/PycharmProjects/MSE/mse_alloc/'
# PosS
spectro = 'HR'
# Targets
target_file = 'mse_sdss_030+62.fit'
mag = [19.5, 20, 'umag']

# Open and display AAO PosS

In [3]:
# read AAO file (coordinates in mm)
poss = Table.read(path + 'sphinx.tsv', format='ascii')
# convert into arcsec
poss['col1'] /= 106.7e-3
poss['col2'] /= 106.7e-3
patrol = 1.24 * 7.77 / 106.7e-3
# plot
reset_output(), output_notebook()
fig1 = figure(title="AAO PosS (LMR)", x_axis_label="RA", y_axis_label="DEC", width=400, height=400)
fig1.circle(poss[poss['col3']=='LR']['col1'], poss[poss['col3']=='LR']['col2'],
            color='red', alpha=0.3, radius=patrol)
fig2 = figure(title="AAO PosS (HR)", x_axis_label="RA", y_axis_label="DEC", width=400, height=400)
fig2.circle(poss[poss['col3']=='HR']['col1'], poss[poss['col3']=='HR']['col2'],
            color='navy', alpha=0.3, radius=patrol)
grid = gridplot([[fig1, fig2]])
show(grid)

Loading BokehJS ...

In [4]:
# Select tiny part of the field for tests
poss_tiny = poss[(np.abs(poss['col1']) < 3500) & (np.abs(poss['col2']) < 3500)]
#poss_tiny = poss

In [5]:
# plot
reset_output(), output_notebook()
fig1 = figure(title="AAO PosS (LMR)", x_axis_label="RA", y_axis_label="DEC",
              width=400, height=400)
fig1.circle(poss_tiny[poss_tiny['col3']=='LR']['col1'],
            poss_tiny[poss_tiny['col3']=='LR']['col2'],
            color='red', alpha=0.3, radius=patrol)
fig2 = figure(title="AAO PosS (HR)", x_axis_label="RA", y_axis_label="DEC",
              width=400, height=400)
fig2.circle(poss_tiny[poss_tiny['col3']=='HR']['col1'],
            poss_tiny[poss_tiny['col3']=='HR']['col2'],
            color='navy', alpha=0.3, radius=patrol)
grid = gridplot([[fig1, fig2]])
show(grid)

Loading BokehJS ...

In [6]:
# Select PosS type and create a Table
fib = poss_tiny[poss_tiny['col3']==spectro]
fib = Table([fib['col1'], fib['col2']], names=('xpos', 'ypos'))

# Create file the user would provide

In [7]:
targets0 = Table.read(path + target_file)

In [8]:
targets1 = targets0[(targets0[mag[2]] >= mag[0]) & (targets0['umag'] <= mag[1])
                   & np.isfinite(targets0['umag'])
                   & np.isfinite(targets0['gmag'])
                   & np.isfinite(targets0['rmag'])
                   & np.isfinite(targets0['imag'])
                   & np.isfinite(targets0['zmag'])]
len(targets0), len(targets1)
#targets1

(1380845, 25360)

In [9]:
targets = Table([targets1['RAJ2000'],
                 targets1['DEJ2000'],
                 targets1['umag'],
                 targets1['gmag'],
                 targets1['rmag'],
                 targets1['imag'],
                 targets1['zmag']],
                names=('RAJ2000', 'DEJ2000', 'umag', 'gmag', 'rmag', 'imag', 'zmag'))
#targets

### Create fake FITS to project catalog onto FoV

In [10]:
# Create fake FITS
im = np.empty([360, 360])
hdu = fits.PrimaryHDU(im)
hdul = fits.HDUList([hdu])
hdul.writeto(path + 'targets.fits', overwrite=True)

In [11]:
# Create default header
hdr = hdul[0].header
hdr['CTYPE1'] = 'RA---TAN'
hdr['CTYPE2'] = 'DEC--TAN'
hdr['CRPIX1'] = 0.
hdr['CRPIX2'] = 0.
hdr['CDELT1'] = -1./3600
hdr['CDELT2'] = 1./3600

# Add coordinates (depends on input file)
hdr['CRVAL1'] = 28.5
hdr['CRVAL2'] = 62.65
#hdul[0].header

In [12]:
# Get pixel coordinates for all targets
wcs0 = wcs.WCS(hdul[0])
xy = [wcs0.wcs_world2pix([[targets['RAJ2000'][i], targets['DEJ2000'][i]]], 1) for i in range(len(targets['RAJ2000']))]
# Make those columns
xpos = Column([xy[i][0][0] for i in range(len(xy))], name='xpos', unit='arcsec')
ypos = Column([xy[i][0][1] for i in range(len(xy))], name='ypos', unit='arcsec')
# Add columns (or replace colums) in table
if 'xpos' in targets.colnames:
    targets['xpos'] = xpos
    targets['ypos'] = ypos
else:
    targets.add_columns([xpos, ypos])

### Add Priority and Repeat columns

Magnitudes are not relevant for the tool, but for this test, we want to use them as the priority rating.

In [13]:
# Using Gmag as the priority rating in this example
prio = np.array([np.round( targets['gmag'][i] - 17 ) for i in range(len(targets))])
prio[prio <= 1] = 1
prio = Column(prio, name='priority')
if 'priority' in targets.colnames:
    targets['priority'] = prio
else:
    targets.add_column(prio)

In [14]:
# Using 1 repeat for all sources (so as to check if the final magnitude distribution is skewed or not)
repeat = np.ones(len(targets))
repeat = Column(repeat, name='repeat')
if 'repeat' in targets.colnames:
    targets['repeat'] = repeat
else:
    targets.add_column(repeat)

### Select targets near FoV and create user file

In [16]:
# Now select only those in the hexagonal field of view
targets_fov = targets[( abs(targets['ypos']) <= 1.0*3600*np.sqrt(3)/2 )
                      & ( abs(targets['xpos']) <= 1.0*3600 )
                      & ( abs(targets['xpos']) <= targets['ypos']/np.sqrt(3) + 1.0*3600 )
                      & ( abs(targets['xpos']) <= -targets['ypos']/np.sqrt(3) + 1.0*3600 )]
# Remove Xpos and Ypos, which should not be in the user-provided file
del targets_fov['xpos']
del targets_fov['ypos']
# Write to an ascii table
targets_fov.write(path + 'targets.dat', format='ascii.ipac', overwrite=True)
# Read it back
targets_fov = Table.read(path + 'targets.dat', format='ascii.ipac')
print(len(targets), len(targets_fov))
#targets_fov

25360 4878


# Process targets provided by user
This is repeating most of what is in the previous section, but before we did it to select the right piece of the massive SDSS or CFHT-LS files ...
Hereafter, we do what the simulator will actually do given the file from the user.

In [204]:
# Read target file
targets = Table.read('targets.dat', format='ascii.ipac')

# Create fake FITS
im = np.empty([360, 360])
hdu = fits.PrimaryHDU(im)
hdul = fits.HDUList([hdu])
# Create default header
hdr = hdul[0].header
hdr['CTYPE1'] = 'RA---TAN'
hdr['CTYPE2'] = 'DEC--TAN'
hdr['CRPIX1'] = 0.
hdr['CRPIX2'] = 0.
hdr['CDELT1'] = -1./3600
hdr['CDELT2'] = 1./3600

# Add coordinates (depends on input file)
hdr['CRVAL1'] = np.mean(targets['RAJ2000'])
hdr['CRVAL2'] = np.mean(targets['DEJ2000'])

# Get pixel coordinates for all targets
wcs0 = wcs.WCS(hdul[0])
xy = [wcs0.wcs_world2pix([[targets['RAJ2000'][i], targets['DEJ2000'][i]]], 1)
      for i in range(len(targets['RAJ2000']))]
# Make those columns
xpos = Column([xy[i][0][0] for i in range(len(xy))], name='xpos', unit='arcsec')
ypos = Column([xy[i][0][1] for i in range(len(xy))], name='ypos', unit='arcsec')
# Add columns to table
targets.add_columns([xpos, ypos])

# Tweak target file for tests
The file is now properly processed, with the right XPOS, YPOS ... but those might not be helpful to run tests, so now is the time to save the targets in a file and change the XPOS, YPOS values

In [80]:
# Don't write it again or I'll have to edit all of it again by hand!!!
# targets.write(path + 'targets_test.dat', format='ascii.ipac', overwrite=True)

In [81]:
targets = Table.read(path + 'targets_test.dat', format='ascii.ipac')

# Plot PosS and targets

In [206]:
# Plot field of view
# PosS
fig1 = figure(title="AAO PosS ("+spectro+")", x_axis_label="RA", y_axis_label="DEC",
              width=400, height=400)
fig1.circle(fib['xpos'], fib['ypos'], color='red', alpha=0.3, radius=patrol)
# Targets
fig1.x(targets['xpos'], targets['ypos'], color='black', alpha=0.1)
# plot hexagonal field of view
fig1.line(3600 * 0.76 * np.array([0.5, 1, 0.5, -0.5, -1, -0.5, 0.5]),
          3600 * 0.76 * np.array([np.sqrt(3.)/2, 0, -np.sqrt(3.)/2, -np.sqrt(3.)/2, 0, np.sqrt(3.)/2, np.sqrt(3.)/2]),
          color='black', line_width=2)
# Show plot
show(fig1)

In [207]:
# Plot magnitude distributions
fig1 = figure(title='Magnitude distributions', x_axis_label='Magnitude')
#u-band
uHist, uEdges = np.histogram(targets['umag'], bins=45, range=[10,25])
fig1.quad(top=uHist, bottom=0, left=uEdges[:-1], right=uEdges[1:],
          fill_color="purple", line_color="darkviolet", alpha=0.5, legend='Umag')
#g-band
gHist, gEdges = np.histogram(targets['gmag'], bins=45, range=[10,25])
fig1.quad(top=gHist, bottom=0, left=gEdges[:-1], right=gEdges[1:],
          fill_color="blue", line_color="darkblue", alpha=0.5, legend='Gmag')
#r-band
rHist, rEdges = np.histogram(targets['rmag'], bins=45, range=[10,25])
fig1.quad(top=rHist, bottom=0, left=rEdges[:-1], right=rEdges[1:],
          fill_color="forestgreen", line_color="green", alpha=0.5, legend='Rmag')
#i-band
iHist, iEdges = np.histogram(targets['imag'], bins=45, range=[10,25])
fig1.quad(top=iHist, bottom=0, left=iEdges[:-1], right=iEdges[1:],
          fill_color="orange", line_color="darkorange", alpha=0.5, legend='Imag')
#z-band
zHist, zEdges = np.histogram(targets['zmag'], bins=45, range=[10,25])
fig1.quad(top=zHist, bottom=0, left=zEdges[:-1], right=zEdges[1:],
          fill_color="red", line_color="darkred", alpha=0.5, legend='Zmag')

show(fig1)

# Compute distances

Now we have all fibers and targets positions, as well as priorities and repeat desired

In [208]:
# Put all distances at 666 to begin with
distances = np.ones([len(targets), len(fib)]) * 666
# Loop over fibers
for f in range(len(fib)):
    # only select targets that are within a small box (+/-10mm) around the fiber
    near = ((np.abs(targets['xpos'] - fib['xpos'][f]) <= 10/106.7e-3)
            & (np.abs(targets['ypos'] - fib['ypos'][f]) <= 10/106.7e-3))
    targets_near = targets[near]
    if len(targets_near) > 0:
        distances[near,f] = [np.sqrt((targets_near['xpos'][i] - fib['xpos'][f])**2
                                     + (targets_near['ypos'][i] - fib['ypos'][f])**2)
                             for i in range(len(targets_near))]
        #print(f, distances[near,f])
distances[distances > (1.24 * 7.77 / 106.7e-3)] = 666

In [209]:
# Plot distribution of distances
fig2 = figure(title='Fiber/Target distances', x_axis_label='Distance (arcsec)')
dHist, dEdges = np.histogram(distances, bins=30, range=[0,10/106.7e-3])
fig2.quad(top=dHist, bottom=0, left=dEdges[:-1], right=dEdges[1:], alpha=0.5)
show(fig2)

# Optimization
Variables are: poss_fov, targets_good, distances_good

In [210]:
# Variables for all inputs (i.e. all targets provided by user and all fibers in PosS)
tgt_all = copy.deepcopy(targets)
fib_all = copy.deepcopy(fib)
dist_all = copy.deepcopy(distances)

# Remove targets that cannot be reached by any fiber
sel = (np.min(distances, axis=1) != 666)
tgt_sel = tgt_all[sel]
dist_sel = distances[sel,:]
print(len(tgt_sel), " targets that can be reached by a positioner out of ", len(tgt_all), "targets")
# Remove fibers that cannot reach any target
sel = (np.min(dist_sel, axis=0) != 666)
fib_sel = fib[sel]
dist_sel = dist_sel[:,sel]
print(len(fib_sel), " fibers that can reach a target out of ", len(fib_all), "fibers")


# Create new variables (should be handled differently in the real simulator)
tgt = copy.deepcopy(tgt_sel)
fib = copy.deepcopy(fib_sel)
dist = copy.deepcopy(dist_sel)
del tgt_sel
del fib_sel
del dist_sel

155  targets that can be reached by a positioner out of  3200 targets
54  fibers that can reach a target out of  56 fibers
